In [7]:
%tensorflow_version 1.x
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os

In [25]:
base_dir = 'drive/My Drive/colorization'
prototxt_old = os.path.join(base_path, 'caffemodels', 'colorization_deploy_v1.prototxt')
caffemodel_old = os.path.join(base_path, 'caffemodels', 'colorization_release_v1.caffemodel')
prototxt_reference = os.path.join(base_path, 'caffemodels', 'reference.prototxt')
caffemodel_reference = os.path.join(base_dir, 'caffemodels', 'reference_model.caffemodel')
reference_pkl = os.path.join(base_dir, 'pretrainedModels', 'reference_model.pkl')
old_pkl = os.path.join(base_dir, 'pretrainedModels', 'old_model.pkl')
prototxt_v2 = os.path.join(base_path, 'caffemodels', 'colorization_deploy_v2.prototxt')
caffemodel_v2 = os.path.join(base_path, 'caffemodels', 'colorization_release_v2.caffemodel')
v2_pikle = os.path.join(base_dir, 'pretrainedModels', 'old_model_v2.pkl')

In [10]:
!apt install caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  caffe-tools-cpu cython3 fonts-lyx javascript-common libblosc1 libcaffe-cpu1
  libgflags2.2 libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5
  liblmdb0 python-matplotlib-data python-tables-data python3-bs4
  python3-caffe-cpu python3-chardet python3-cycler python3-dateutil
  python3-decorator python3-gflags python3-h5py python3-html5lib
  python3-ipython python3-ipython-genutils python3-leveldb python3-lxml
  python3-matplotlib python3-networkx python3-nose python3-numexpr
  python3-olefile python3-pandas python3-pandas-lib python3-pexpect
  python3-pickleshare python3-pil python3-pkg-resources python3-prompt-toolkit
  python3-protobuf python3-ptyprocess python3-pygments python3-pyparsing
  pyt

In [11]:
import caffe

#https://stackoverflow.com/questions/45199643/how-do-i-load-a-caffe-model-and-convert-to-a-numpy-array

def shai_net_to_py_readable(prototxt_filename, caffemodel_filename):
  net = caffe.Net(prototxt_filename, caffemodel_filename, caffe.TEST) # read the net + weights
  pynet_ = []
  for li in range(len(net.layers)):  # for each layer in the net
    layer = {}  # store layer's information
    layer['name'] = net._layer_names[li]
    # for each input to the layer (aka "bottom") store its name and shape
    layer['bottoms'] = [(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape)
                         for bi in list(net._bottom_ids(li))]
    # for each output of the layer (aka "top") store its name and shape
    layer['tops'] = [(net._blob_names[bi], net.blobs[net._blob_names[bi]].data.shape)
                      for bi in list(net._top_ids(li))]
    layer['type'] = net.layers[li].type  # type of the layer
    # the internal parameters of the layer. not all layers has weights.
    layer['weights'] = [net.layers[li].blobs[bi].data[...]
                        for bi in range(len(net.layers[li].blobs))]
    pynet_.append(layer)
  return pynet_

In [26]:
pynet = shai_net_to_py_readable(prototxt_filename=prototxt_reference, caffemodel_filename=caffemodel_reference)
with open(reference_pkl, "wb") as f:
    pickle.dump(pynet, f)

pynet = shai_net_to_py_readable(prototxt_filename=prototxt_old, caffemodel_filename=caffemodel_old)
with open(old_pkl, "wb") as f:
    pickle.dump(pynet, f)

In [27]:
pynet = shai_net_to_py_readable(prototxt_filename=prototxt_v2, caffemodel_filename=caffemodel_v2)
with open(v2_pikle, "wb") as f:
   pickle.dump(pynet, f)

In [28]:
#Image Colorization 2016 architecture

with open(old_pkl, "rb") as f:
  model = pickle.load(f)

for layer in model:
  #type of layer
  print(layer['type'])
  #input shape
  print(layer['bottoms'])

Input
[]
Convolution
[('data_l', (1, 1, 224, 224))]
ReLU
[('conv1_1', (1, 64, 224, 224))]
Convolution
[('conv1_1', (1, 64, 224, 224))]
ReLU
[('conv1_2', (1, 64, 112, 112))]
BatchNorm
[('conv1_2', (1, 64, 112, 112))]
Convolution
[('conv1_2norm', (1, 64, 112, 112))]
ReLU
[('conv2_1', (1, 128, 112, 112))]
Convolution
[('conv2_1', (1, 128, 112, 112))]
ReLU
[('conv2_2', (1, 128, 56, 56))]
BatchNorm
[('conv2_2', (1, 128, 56, 56))]
Convolution
[('conv2_2norm', (1, 128, 56, 56))]
ReLU
[('conv3_1', (1, 256, 56, 56))]
Convolution
[('conv3_1', (1, 256, 56, 56))]
ReLU
[('conv3_2', (1, 256, 56, 56))]
Convolution
[('conv3_2', (1, 256, 56, 56))]
ReLU
[('conv3_3', (1, 256, 28, 28))]
BatchNorm
[('conv3_3', (1, 256, 28, 28))]
Convolution
[('conv3_3norm', (1, 256, 28, 28))]
ReLU
[('conv4_1', (1, 512, 28, 28))]
Convolution
[('conv4_1', (1, 512, 28, 28))]
ReLU
[('conv4_2', (1, 512, 28, 28))]
Convolution
[('conv4_2', (1, 512, 28, 28))]
ReLU
[('conv4_3', (1, 512, 28, 28))]
BatchNorm
[('conv4_3', (1, 512, 28,

In [29]:
#User-guided Image Colorization 2019 architecture

with open(reference_pkl, "rb") as f:
  model = pickle.load(f)

for layer in model:
  #type of layer
  print(layer['type'])
  #input shape
  print(layer['bottoms'])

Input
[]
Slice
[('data_l_ab_mask', (1, 4, 256, 256))]
Convolution
[('data_ab_mask', (1, 3, 256, 256))]
Convolution
[('data_l', (1, 1, 256, 256))]
Eltwise
[('bw_conv1_1', (1, 64, 256, 256)), ('ab_conv1_1', (1, 64, 256, 256))]
ReLU
[('conv1_1', (1, 64, 256, 256))]
Convolution
[('conv1_1', (1, 64, 256, 256))]
ReLU
[('conv1_2', (1, 64, 256, 256))]
BatchNorm
[('conv1_2', (1, 64, 256, 256))]
Split
[('conv1_2norm', (1, 64, 256, 256))]
Convolution
[('conv1_2norm_conv1_2norm_0_split_0', (1, 64, 256, 256))]
Convolution
[('conv1_2norm_ss', (1, 64, 128, 128))]
ReLU
[('conv2_1', (1, 128, 128, 128))]
Convolution
[('conv2_1', (1, 128, 128, 128))]
ReLU
[('conv2_2', (1, 128, 128, 128))]
BatchNorm
[('conv2_2', (1, 128, 128, 128))]
Split
[('conv2_2norm', (1, 128, 128, 128))]
Convolution
[('conv2_2norm_conv2_2norm_0_split_0', (1, 128, 128, 128))]
Convolution
[('conv2_2norm_ss', (1, 128, 64, 64))]
ReLU
[('conv3_1', (1, 256, 64, 64))]
Convolution
[('conv3_1', (1, 256, 64, 64))]
ReLU
[('conv3_2', (1, 256, 64

In [30]:
with open(old_pkl, "rb") as f:
  old_model = pickle.load(f)

types = {}

for layer in old_model:
  layer_type = layer['type']
  if layer_type in types:
    types[layer_type] = types[layer_type] + 1
  else:
    types[layer_type] = 1

print(types)

{'Input': 1, 'Convolution': 23, 'ReLU': 22, 'BatchNorm': 7, 'Deconvolution': 1, 'Eltwise': 1, 'Softmax': 1, 'Silence': 1}


In [31]:
def getWeights(model):
  weights = {}
  biases = {}
  weight_shapes = {}
  bias_shapes = {}

  for layer in model:
    count = len(layer['weights'])
    if count == 2:
      layer_weight = layer['weights'][0]
      layer_bias = layer['weights'][1]
      name = layer['tops'][0][0]
      weights[name] = layer_weight
      biases[name] = layer_bias
      weight_shapes[name] = layer_weight.shape
      bias_shapes[name] = layer_bias.shape
    elif count == 3:
      layer_weight1 = layer['weights'][0]
      layer_weight2 = layer['weights'][1]
      layer_weight3 = layer['weights'][2]
      name = layer['tops'][0][0]
      weights[name + '_1'] = layer_weight1
      weights[name + '_2'] = layer_weight2
      weights[name + '_3'] = layer_weight3
      weight_shapes[name + '_1'] = layer_weight1.shape
      weight_shapes[name + '_2'] = layer_weight2.shape
      weight_shapes[name + '_3'] = layer_weight3.shape
  return weights, biases, weight_shapes, bias_shapes

In [32]:
weights_old, biases_old, weight_shapes_old, bias_shapes_old = getWeights(old_model)

In [33]:
old_weights = os.path.join(base_dir, 'pretrainedWeights', 'weights_old.pkl')
old_biases = os.path.join(base_dir, 'pretrainedWeights', 'biases_old.pkl')
old_weight_shapes = os.path.join(base_dir, 'pretrainedWeights', 'weight_shapes_old.pkl')
old_bias_shapes = os.path.join(base_dir, 'pretrainedWeights', 'bias_shapes_old.pkl')

with open(old_weights, 'wb') as f:
  pickle.dump(weights_old, f)

with open(old_biases, 'wb') as f:
  pickle.dump(biases_old, f)

with open(old_weight_shapes, 'wb') as f:
  pickle.dump(weight_shapes_old, f)

with open(old_bias_shapes, 'wb') as f:
  pickle.dump(bias_shapes_old, f)

In [34]:
with open(reference_pkl, "rb") as f:
  new_model = pickle.load(f)

types = {}

for layer in new_model:
  layer_type = layer['type']
  if layer_type in types:
    types[layer_type] = types[layer_type] + 1
  else:
    types[layer_type] = 1

print(types)

{'Input': 1, 'Slice': 1, 'Convolution': 31, 'Eltwise': 4, 'ReLU': 26, 'BatchNorm': 9, 'Split': 3, 'Deconvolution': 3, 'TanH': 1, 'Scale': 1}


In [35]:
weights_new, biases_new, weight_shapes_new, bias_shapes_new = getWeights(new_model)

In [36]:
print(weight_shapes_new)
print(weight_shapes_old)

{'ab_conv1_1': (64, 3, 3, 3), 'bw_conv1_1': (64, 1, 3, 3), 'conv1_2': (64, 64, 3, 3), 'conv1_2norm_1': (64,), 'conv1_2norm_2': (64,), 'conv1_2norm_3': (1,), 'conv2_1': (128, 64, 3, 3), 'conv2_2': (128, 128, 3, 3), 'conv2_2norm_1': (128,), 'conv2_2norm_2': (128,), 'conv2_2norm_3': (1,), 'conv3_1': (256, 128, 3, 3), 'conv3_2': (256, 256, 3, 3), 'conv3_3': (256, 256, 3, 3), 'conv3_3norm_1': (256,), 'conv3_3norm_2': (256,), 'conv3_3norm_3': (1,), 'conv4_1': (512, 256, 3, 3), 'conv4_2': (512, 512, 3, 3), 'conv4_3': (512, 512, 3, 3), 'conv4_3norm_1': (512,), 'conv4_3norm_2': (512,), 'conv4_3norm_3': (1,), 'conv5_1': (512, 512, 3, 3), 'conv5_2': (512, 512, 3, 3), 'conv5_3': (512, 512, 3, 3), 'conv5_3norm_1': (512,), 'conv5_3norm_2': (512,), 'conv5_3norm_3': (1,), 'conv6_1': (512, 512, 3, 3), 'conv6_2': (512, 512, 3, 3), 'conv6_3': (512, 512, 3, 3), 'conv6_3norm_1': (512,), 'conv6_3norm_2': (512,), 'conv6_3norm_3': (1,), 'conv7_1': (512, 512, 3, 3), 'conv7_2': (512, 512, 3, 3), 'conv7_3': (512

In [37]:
new_weights = 'drive/My Drive/colorization/pretrainedWeights/weights_new.pkl'
new_biases = 'drive/My Drive/colorization/pretrainedWeights/biases_new.pkl'
new_weight_shapes = 'drive/My Drive/colorization/pretrainedWeights/weight_shapes_new.pkl'
new_bias_shapes = 'drive/My Drive/colorization/pretrainedWeights/bias_shapes_new.pkl'

with open(new_weights, 'wb') as f:
  pickle.dump(weights_new, f)

with open(new_biases, 'wb') as f:
  pickle.dump(biases_new, f)

with open(new_weight_shapes, 'wb') as f:
  pickle.dump(weight_shapes_new, f)

with open(new_bias_shapes, 'wb') as f:
  pickle.dump(bias_shapes_new, f)

In [38]:
with open(v2_pikle, "rb") as f:
  model = pickle.load(f)

for layer in model:
  print(layer['type'])
  print(layer['bottoms'])

Input
[]
Convolution
[('data_l', (1, 1, 224, 224))]
ReLU
[('conv1_1', (1, 64, 224, 224))]
Convolution
[('conv1_1', (1, 64, 224, 224))]
ReLU
[('conv1_2', (1, 64, 112, 112))]
BatchNorm
[('conv1_2', (1, 64, 112, 112))]
Convolution
[('conv1_2norm', (1, 64, 112, 112))]
ReLU
[('conv2_1', (1, 128, 112, 112))]
Convolution
[('conv2_1', (1, 128, 112, 112))]
ReLU
[('conv2_2', (1, 128, 56, 56))]
BatchNorm
[('conv2_2', (1, 128, 56, 56))]
Convolution
[('conv2_2norm', (1, 128, 56, 56))]
ReLU
[('conv3_1', (1, 256, 56, 56))]
Convolution
[('conv3_1', (1, 256, 56, 56))]
ReLU
[('conv3_2', (1, 256, 56, 56))]
Convolution
[('conv3_2', (1, 256, 56, 56))]
ReLU
[('conv3_3', (1, 256, 28, 28))]
BatchNorm
[('conv3_3', (1, 256, 28, 28))]
Convolution
[('conv3_3norm', (1, 256, 28, 28))]
ReLU
[('conv4_1', (1, 512, 28, 28))]
Convolution
[('conv4_1', (1, 512, 28, 28))]
ReLU
[('conv4_2', (1, 512, 28, 28))]
Convolution
[('conv4_2', (1, 512, 28, 28))]
ReLU
[('conv4_3', (1, 512, 28, 28))]
BatchNorm
[('conv4_3', (1, 512, 28,

In [ ]:
w1 = tf.Variable(pynet[0]['weights'], name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
#.meta - the complete tf graph
#.data - contains the training variables
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#write_meta_graph=False - don't write the graph on every iteration, it will not change
saver.save(sess, "drive/My Drive/my_test_model", global_step=1000, write_meta_graph=False)

'drive/My Drive/my_test_model-1000'

In [ ]:
#values of placeholders are not saved
with tf.Session() as sess:
  #import saved model
  new_saver = tf.train.import_meta_graph('my_test_model-1000.meta')
  #restore values of variables
  new_saver.restore(sess, tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./my_test_model-1000
